### Below Codes use older version packages to support Udacity workspace

In [2]:
# %env OPENAI_API_KEY=Your API Key

In [3]:
import os
import json
import textwrap
import uuid
from typing import Optional, List, Dict

import pandas as pd
import openai
from pydantic import BaseModel

import chromadb
from chromadb.config import Settings

In [4]:
# -------------------------
# CONFIG
# -------------------------
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise RuntimeError("Please set OPENAI_API_KEY")

openai.api_key = OPENAI_API_KEY
openai.api_base = "https://openai.vocareum.com/v1"  # IMPORTANT

CHAT_MODEL = "gpt-3.5-turbo"
EMBEDDING_MODEL = "text-embedding-ada-002"


# -------------------------
# HELPERS
# -------------------------
def gpt_chat(prompt, temperature=0.0, print_output=True):
    print("LLM Started....")
    resp = openai.ChatCompletion.create(
        model=CHAT_MODEL,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature,
    )
    out = resp.choices[0].message["content"].strip()
    if print_output:
        print(out)
    return out


def get_embeddings(texts: List[str]):
    resp = openai.Embedding.create(model=EMBEDDING_MODEL, input=texts)
    return [d["embedding"] for d in resp["data"]]

In [5]:
def generate_homes_csv(out_path="homes.csv"):
    headers = "Neighborhood,Price,Bedroom,Bathroom,House_size,Description"

    prompt = f"""
Generate 10 synthetic house listings in strict CSV format.
Headers: {headers}

Rules:
1. Only header + 10 rows
2. No commas in numbers
3. Description must NOT contain commas
4. All fields in double quotes
"""

    csv_text = gpt_chat(prompt)
    with open(out_path, "w", encoding="utf-8") as f:
        f.write(csv_text)

In [6]:
# -------------------------
# DATA PROCESSING
# -------------------------
def row_to_text(row):
    return (
        f"{row['Neighborhood']} "
        f"{row['Price']} "
        f"{row['Bedroom']} Bedroom "
        f"{row['Bathroom']} Bathroom "
        f"{row['House_size']} "
        f"{row['Description']}"
    )


# -------------------------
# USER PREF MODEL (Pydantic v1)
# -------------------------
class Prefs(BaseModel):
    budget: Optional[int] = None
    bedroom: Optional[int] = None
    bathroom: Optional[int] = None
    min_size: Optional[int] = None
    preferances: Optional[str] = None


# ---------------------------------------------------------
# EXECUTION STARTS HERE (NO main(), script runs top to bottom)
# ---------------------------------------------------------

# 1) Generate CSV
generate_homes_csv("homes.csv")

# 2) Load CSV
df = pd.read_csv("homes.csv")
print(df.head())

# 3) Build vectors
documents = []
metadatas = []
ids = []

for _, row in df.iterrows():
    documents.append(row_to_text(row))
    metadatas.append({
        "Neighborhood": str(row["Neighborhood"]),
        "Price": int(row["Price"]),
        "Bedroom": int(row["Bedroom"]),
        "Bathroom": int(row["Bathroom"]),
        "House_size": int(row["House_size"]),
        "Description": str(row["Description"]),
    })
    ids.append(str(uuid.uuid4()))

LLM Started....
Neighborhood,Price,Bedroom,Bathroom,House_size,Description
"Midtown",250000,3,2,2000,"Beautiful single family home with spacious backyard"
"Downtown",350000,4,3,3000,"Luxurious townhouse with rooftop terrace and city views"
"West End",180000,2,1,1500,"Cozy bungalow with updated kitchen and hardwood floors"
"East Side",400000,5,4,3500,"Modern mansion with pool, guest house, and tennis court"
"North Hills",280000,3,2,2200,"Charming colonial style house with wrap-around porch"
"South End",220000,2,1,1600,"Renovated ranch with open floor plan and vaulted ceilings"
"Old Town",320000,4,3,2800,"Historic Victorian home with original details and stained glass windows"
"Waterfront",500000,6,5,4000,"Waterfront estate with private dock and panoramic ocean views"
"Suburbia",200000,3,2,1800,"Family-friendly home in quiet neighborhood with top-rated schools"
"Mountain View",450000,5,4,3800,"Custom-built log cabin with stone fireplace and mountain views"
  Neighborhood   Price  Bedroom

In [7]:
# 4) Embeddings
print("Computing embeddings...")
embeddings = get_embeddings(documents)

# 5) Chroma vector DB
client = chromadb.Client(Settings())
name = "homes_vdb"

try:
    client.delete_collection(name)
except:
    pass

collection = client.create_collection(name)

print("Adding to Chroma...")
collection.add(
    ids=ids,
    metadatas=metadatas,
    documents=documents,
    embeddings=embeddings,
)

print("Vector DB ready!")

Computing embeddings...
Adding to Chroma...
Vector DB ready!


In [8]:
# 6) User preferences extraction
example = "3 bedroom and 2 bathroom under $500000 with minimum 100 sqft with a mountain view"
user_text = input(f"Enter preferences (ex: {example}): ").strip()
if not user_text:
    user_text = example

extract_prompt = f"""
Extract JSON from:

\"\"\"{user_text}\"\"\"

Format:
{{
 "budget": int|null,
 "bedroom": int|null,
 "bathroom": int|null,
 "min_size": int|null,
 "preferances": string|null
}}

Return ONLY JSON.
"""

prefs_raw = gpt_chat(extract_prompt)

try:
    first = prefs_raw.find("{")
    last = prefs_raw.rfind("}")
    prefs = Prefs(**json.loads(prefs_raw[first:last+1]))
except:
    prefs = Prefs()

# Ask for missing fields
missing = [k for k, v in prefs.dict().items() if v is None and k != "preferances"]

if missing:
    print("Missing:", missing)
    for m in missing:
        val = input(f"Enter {m} (blank to skip): ").strip()
        if val:
            try:
                setattr(prefs, m, int(val))
            except:
                pass

print("\nFinal Prefs:", prefs.dict())

Enter preferences (ex: 3 bedroom and 2 bathroom under $500000 with minimum 100 sqft with a mountain view): 3 bedroom and 2 bathroom under $500000 with a water view
LLM Started....
{
 "budget": 500000,
 "bedroom": 3,
 "bathroom": 2,
 "min_size": null,
 "preferances": "water view"
}
Missing: ['min_size']
Enter min_size (blank to skip): 

Final Prefs: {'budget': 500000, 'bedroom': 3, 'bathroom': 2, 'min_size': None, 'preferances': 'water view'}


In [9]:
# 7) Prepare query
query_text = " ".join([
    prefs.preferances or "",
    f"{prefs.bedroom} bedroom" if prefs.bedroom else "",
    f"{prefs.bathroom} bathroom" if prefs.bathroom else "",
    f"{prefs.budget} budget" if prefs.budget else "",
    f"{prefs.min_size} sqft" if prefs.min_size else "",
]).strip()

if not query_text:
    query_text = "house listing"

print("\nSearching for:", query_text)

query_emb = get_embeddings([query_text])[0]

results = collection.query(
    query_embeddings=[query_emb],
    n_results=10,
    include=["documents", "metadatas", "distances"]
)

metas = results["metadatas"][0]
docs = results["documents"][0]
dists = results["distances"][0]

filtered = []
for md, doc, dist in zip(metas, docs, dists):
    price = int(md["Price"])
    size = int(md["House_size"])
    bed = int(md["Bedroom"])
    bath = int(md["Bathroom"])

    ok = True
    if prefs.budget and price > prefs.budget:
        ok = False
    if prefs.bedroom and bed < prefs.bedroom:
        ok = False
    if prefs.bathroom and bath < prefs.bathroom:
        ok = False
    if prefs.min_size and size < prefs.min_size:
        ok = False

    if ok:
        filtered.append({"metadata": md, "doc": doc, "dist": dist})


Searching for: water view 3 bedroom 2 bathroom 500000 budget


In [10]:
print("\n---- TOP MATCHES ----")
for i, r in enumerate(filtered[:3], 1):
    print(f"\n🏠 Home {i}")
    for k, v in r["metadata"].items():
        print(f"{k}: {v}")

if not filtered:
    print("No matches found!")
    exit()

# 8) Recommendation
best = filtered[0]["metadata"]

rec_prompt = f"""
Create a short friendly home recommendation with emojis.

User:
{json.dumps(prefs.dict(), indent=2)}

Home:
{json.dumps(best, indent=2)}
"""

recommendation = gpt_chat(rec_prompt, temperature=0.7)

print("\n🎯 Personalized Recommendation:\n")
print(textwrap.fill(recommendation, width=100))



---- TOP MATCHES ----

🏠 Home 1
Bathroom: 5
Bedroom: 6
Description: Waterfront estate with private dock and panoramic ocean views
House_size: 4000
Neighborhood: Waterfront
Price: 500000

🏠 Home 2
Bathroom: 3
Bedroom: 4
Description: Luxurious townhouse with rooftop terrace and city views
House_size: 3000
Neighborhood: Downtown
Price: 350000

🏠 Home 3
Bathroom: 4
Bedroom: 5
Description: Modern mansion with pool, guest house, and tennis court
House_size: 3500
Neighborhood: East Side
Price: 400000
LLM Started....
🏡 Looking for a spacious waterfront home with breathtaking ocean views? Look no further! This 6 bedroom, 5 bathroom estate comes with a private dock and panoramic ocean views, perfect for those who love the water. Priced at $500,000, this home is a steal! Don't miss out on this amazing opportunity! 🌊🏝️ #dreamhome #waterfrontliving

🎯 Personalized Recommendation:

🏡 Looking for a spacious waterfront home with breathtaking ocean views? Look no further! This 6
bedroom, 5 bathroom es